In [1]:
from __future__ import annotations

import argparse
import re
from pathlib import Path

from dotenv import load_dotenv
import os
import json
import requests
from typing import Dict, Any, Iterable, Optional, Iterator, List ,Tuple

from dataclasses import dataclass
import hashlib

from pypdf import PdfReader

import json
import numpy as np
from tqdm import tqdm

import faiss
from sentence_transformers import SentenceTransformer

PROJECT_ROOT = Path().resolve()
DATA_DIR  = PROJECT_ROOT / "data"
CACHE_DIR = PROJECT_ROOT / "cache"

### 1 - Chargement de documents (.txt / .pdf) avec métadonnées + hash SHA256

In [2]:
@dataclass
class Document:
    text: str
    meta: Dict[str, Any]

def sha256_file(path: Path) -> str:
    h = hashlib.sha256()
    with path.open("rb") as f:
        for chunk in iter(lambda: f.read(1024 * 1024), b""):
            h.update(chunk)
    return h.hexdigest()

def iter_files(data_dir: Path) -> Iterator[Path]:
    for p in data_dir.rglob("*"):
        if p.is_file() and p.suffix.lower() in {".txt", ".pdf"}:
            yield p

def load_txt(path: Path) -> List[Document]:
    txt = path.read_text(encoding="utf-8", errors="ignore")
    return [Document(text=txt, meta={"path": str(path), "page": None})]

def load_pdf(path: Path) -> List[Document]:
    docs: List[Document] = []
    reader = PdfReader(str(path))
    for i, page in enumerate(reader.pages):
        try:
            text = page.extract_text() or ""
        except Exception:
            text = ""
        if text.strip():
            docs.append(Document(text=text, meta={"path": str(path), "page": i + 1}))
    return docs

def load_any(path: Path) -> List[Document]:
    if path.suffix.lower() == ".txt":
        return load_txt(path)
    if path.suffix.lower() == ".pdf":
        return load_pdf(path)
    return []


#### Définition (points clairs)

* Définit une structure `Document` (dataclass) :

  * `text` : contenu texte
  * `meta` : métadonnées (ex: chemin, page)
* `sha256_file(path)` :

  * Calcule l’empreinte **SHA-256** d’un fichier (lecture par chunks 1MB) pour identifier un fichier de façon unique (utile pour cache/version).
* `iter_files(data_dir)` :

  * Parcourt récursivement un dossier et renvoie uniquement les fichiers **.txt** et **.pdf**.
* `load_txt(path)` :

  * Lit un fichier texte en UTF-8 (ignore erreurs) et retourne une liste contenant **un seul Document** (page = `None`).
* `load_pdf(path)` :

  * Lit un PDF page par page, extrait le texte, et crée un `Document` par page non vide avec `page = i+1`.
* `load_any(path)` :

  * Routeur simple : appelle `load_txt` ou `load_pdf` selon l’extension, sinon renvoie une liste vide.

### 2 - Construction d’un index RAG (chunking + embeddings + cache + FAISS)

In [3]:
def chunk_text(text: str, chunk_size: int = 900, overlap: int = 150) -> List[str]:
    text = " ".join(text.split())
    if not text:
        return []
    chunks = []
    start = 0
    n = len(text)
    while start < n:
        end = min(n, start + chunk_size)
        chunks.append(text[start:end])
        if end == n:
            break
        start = max(0, end - overlap)
    return chunks

@dataclass
class Chunk:
    text: str
    meta: Dict[str, Any]  # path, page, chunk_id

@dataclass
class RAGIndex:
    index: faiss.Index
    chunks: List[Chunk]         # position i -> chunk
    dim: int

def _ensure_dirs(cache_dir: Path):
    (cache_dir / "chunks").mkdir(parents=True, exist_ok=True)
    (cache_dir / "embeddings").mkdir(parents=True, exist_ok=True)

def build_or_load_index(
    data_dir: Path,
    cache_dir: Path,
    embedding_model_name: str = "sentence-transformers/all-MiniLM-L6-v2",
    chunk_size: int = 900,
    overlap: int = 150,
) -> RAGIndex:
    """
    Cache par fichier:
    - cache/chunks/<hash>.jsonl
    - cache/embeddings/<hash>.npy
    + cache/file_hashes.json (mapping path -> hash)
    On rebuild l'index FAISS à partir des caches.
    """
    _ensure_dirs(cache_dir)

    hashes_path = cache_dir / "file_hashes.json"
    old_hashes: Dict[str, str] = {}
    if hashes_path.exists():
        old_hashes = json.loads(hashes_path.read_text(encoding="utf-8"))

    new_hashes: Dict[str, str] = {}
    files = sorted(list(iter_files(data_dir)))

    # Embedder
    embedder = SentenceTransformer(embedding_model_name)

    all_chunks: List[Chunk] = []
    all_embs: List[np.ndarray] = []

    for fp in tqdm(files, desc="Scan & Cache"):
        file_hash = sha256_file(fp)
        new_hashes[str(fp)] = file_hash

        chunks_file = cache_dir / "chunks" / f"{file_hash}.jsonl"
        emb_file = cache_dir / "embeddings" / f"{file_hash}.npy"

        need_recompute = (old_hashes.get(str(fp)) != file_hash) or (not chunks_file.exists()) or (not emb_file.exists())

        if need_recompute:
            docs: List[Document] = load_any(fp)
            file_chunks: List[Chunk] = []
            for d in docs:
                pieces = chunk_text(d.text, chunk_size=chunk_size, overlap=overlap)
                for j, ch in enumerate(pieces):
                    meta = dict(d.meta)
                    meta["chunk_id"] = j
                    file_chunks.append(Chunk(text=ch, meta=meta))

            # save chunks jsonl
            with chunks_file.open("w", encoding="utf-8") as f:
                for c in file_chunks:
                    f.write(json.dumps({"text": c.text, "meta": c.meta}, ensure_ascii=False) + "\n")

            # embed + normalize for cosine
            if file_chunks:
                embs = embedder.encode([c.text for c in file_chunks], show_progress_bar=False, convert_to_numpy=True)
                embs = embs.astype("float32")
                faiss.normalize_L2(embs)
            else:
                embs = np.zeros((0, embedder.get_sentence_embedding_dimension()), dtype="float32")

            np.save(emb_file, embs)
        # load cached
        file_chunks_loaded: List[Chunk] = []
        with chunks_file.open("r", encoding="utf-8") as f:
            for line in f:
                obj = json.loads(line)
                file_chunks_loaded.append(Chunk(text=obj["text"], meta=obj["meta"]))
        embs_loaded = np.load(emb_file).astype("float32")

        # align safety
        if len(file_chunks_loaded) != embs_loaded.shape[0]:
            # fallback: recompute quickly (rare)
            docs = load_any(fp)
            file_chunks_loaded = []
            for d in docs:
                pieces = chunk_text(d.text, chunk_size=chunk_size, overlap=overlap)
                for j, ch in enumerate(pieces):
                    meta = dict(d.meta)
                    meta["chunk_id"] = j
                    file_chunks_loaded.append(Chunk(text=ch, meta=meta))
            if file_chunks_loaded:
                embs_loaded = embedder.encode([c.text for c in file_chunks_loaded], show_progress_bar=False, convert_to_numpy=True).astype("float32")
                faiss.normalize_L2(embs_loaded)
            else:
                embs_loaded = np.zeros((0, embedder.get_sentence_embedding_dimension()), dtype="float32")

        all_chunks.extend(file_chunks_loaded)
        if embs_loaded.size:
            all_embs.append(embs_loaded)

    # save hashes
    hashes_path.write_text(json.dumps(new_hashes, ensure_ascii=False, indent=2), encoding="utf-8")

    dim = embedder.get_sentence_embedding_dimension()
    if all_embs:
        mat = np.vstack(all_embs).astype("float32")
    else:
        mat = np.zeros((0, dim), dtype="float32")

    # FAISS cosine (IP sur vecteurs normalisés)
    index = faiss.IndexFlatIP(dim)
    if mat.shape[0] > 0:
        index.add(mat)

    return RAGIndex(index=index, chunks=all_chunks, dim=dim)

#### Définition (points clairs)

* `chunk_text(...)` :

  * Nettoie le texte (espaces) puis le découpe en **chunks** de taille `chunk_size` avec **overlap** `overlap` (pour garder du contexte entre morceaux).
* Définit 2 structures :

  * `Chunk` : un morceau de texte + métadonnées (`path`, `page`, `chunk_id`)
  * `RAGIndex` : l’index FAISS + la liste des chunks (alignés par position) + dimension des embeddings
* `_ensure_dirs(cache_dir)` :

  * Crée les dossiers cache `cache/chunks` et `cache/embeddings`.
* `build_or_load_index(...)` :

  * Parcourt tous les fichiers `.txt` et `.pdf` dans `data_dir`.
  * Calcule un **hash SHA256** par fichier pour détecter les changements.
  * Utilise un modèle `SentenceTransformer` pour générer des **embeddings** de chaque chunk.
  * **Met en cache** par fichier :

    * `chunks/<hash>.jsonl` (texte + meta)
    * `embeddings/<hash>.npy` (matrice embeddings)
    * `file_hashes.json` (mapping path → hash)
  * Recharge depuis le cache si rien n’a changé, sinon **recalcule** chunks + embeddings.
  * Normalise les embeddings (`faiss.normalize_L2`) pour faire une similarité **cosine**.
  * Construit un index FAISS `IndexFlatIP` (produit scalaire sur vecteurs normalisés = cosine) et ajoute tous les embeddings.
  * Retourne un objet `RAGIndex` prêt pour la recherche.


### 3 - Recherche RAG : embedding de la requête + top-k chunks via FAISS

In [4]:
@dataclass
class Retrieved:
    chunk: Chunk
    score: float
    ref_id: int

class Retriever:
    def __init__(self, rag_index: RAGIndex, embedder_name: str = "sentence-transformers/all-MiniLM-L6-v2"):
        self.rag_index = rag_index
        self.embedder = SentenceTransformer(embedder_name)

    def search(self, query: str, topk: int = 6) -> List[Retrieved]:
        if self.rag_index.index.ntotal == 0:
            return []
        q = self.embedder.encode([query], convert_to_numpy=True).astype("float32")
        faiss.normalize_L2(q)
        scores, ids = self.rag_index.index.search(q, topk)
        out: List[Retrieved] = []
        for rank, (idx, sc) in enumerate(zip(ids[0].tolist(), scores[0].tolist()), start=1):
            if idx < 0 or idx >= len(self.rag_index.chunks):
                continue
            out.append(Retrieved(chunk=self.rag_index.chunks[idx], score=float(sc), ref_id=rank))
        return out

#### Définition (points clairs)

* Définit `Retrieved` :

  * `chunk` : le chunk récupéré
  * `score` : score de similarité (cosine via FAISS IP sur vecteurs normalisés)
  * `ref_id` : numéro de référence (rang 1..topk)
* `Retriever.__init__` :

  * Reçoit un `RAGIndex` (FAISS + chunks) et charge un `SentenceTransformer` pour encoder les requêtes.
* `search(query, topk=6)` :

  * Si l’index est vide → renvoie `[]`.
  * Encode la requête en vecteur, puis normalise (`faiss.normalize_L2`) pour cosine.
  * Interroge FAISS (`index.search`) pour obtenir les `topk` meilleurs ids + scores.
  * Convertit chaque résultat en objet `Retrieved`, en vérifiant que l’id correspond bien à un chunk existant.
  * Retourne une liste ordonnée par rang (ref_id = 1, 2, 3, …).


### 4 - Streaming chat AtlasCloud (style OpenAI) avec clé API + parsing des deltas

In [5]:
ATLAS_URL = "https://api.atlascloud.ai/v1/chat/completions"

def _auth_headers() -> Dict[str, str]:
    key = os.getenv("ATLASCLOUD_API_KEY", "").strip()
    if not key:
        raise RuntimeError("ATLASCLOUD_API_KEY manquante. Mets-la dans .env")
    return {
        "Content-Type": "application/json",
        "Authorization": f"Bearer {key}",
    }

def chat_stream(messages, model: Optional[str] = None, max_tokens: int = 2048, temperature: float = 0.2) -> Iterable[str]:
    """
    Stream type OpenAI: lignes 'data: {...}' + 'data: [DONE]'
    Renvoie les morceaux de texte (delta).
    """
    model = model or os.getenv("ATLAS_MODEL", "openai/gpt-oss-20b")
    payload: Dict[str, Any] = {
        "model": model,
        "messages": messages,
        "max_tokens": max_tokens,
        "temperature": temperature,
        "stream": True,
    }

    with requests.post(ATLAS_URL, headers=_auth_headers(), json=payload, stream=True, timeout=300) as r:
        r.raise_for_status()
        for raw in r.iter_lines(decode_unicode=True):
            if not raw:
                continue
            line = raw.strip()
            # Plusieurs providers envoient parfois du JSON direct.
            if line.startswith("data:"):
                data = line[len("data:"):].strip()
                if data == "[DONE]":
                    break
                try:
                    obj = json.loads(data)
                except json.JSONDecodeError:
                    continue
            else:
                # fallback si pas "data:"
                try:
                    obj = json.loads(line)
                except json.JSONDecodeError:
                    continue

            # Format type OpenAI: choices[0].delta.content
            try:
                delta = obj["choices"][0].get("delta", {})
                content = delta.get("content")
                if content:
                    yield content
            except Exception:
                # fallback: choices[0].message.content (non-stream)
                try:
                    content = obj["choices"][0]["message"]["content"]
                    if content:
                        yield content
                except Exception:
                    continue

#### Définition (points clairs)

* Définit l’endpoint `ATLAS_URL` pour appeler l’API AtlasCloud (`/chat/completions`).
* `_auth_headers()` :

  * Récupère `ATLASCLOUD_API_KEY` depuis l’environnement (`.env`) et construit les headers `Authorization: Bearer ...`.
  * Si la clé manque → lève une erreur claire.
* `chat_stream(messages, ...)` :

  * Construit un payload compatible OpenAI (`model`, `messages`, `max_tokens`, `temperature`, `stream=True`).
  * Envoie une requête `POST` avec `requests.post(..., stream=True)` pour lire la réponse en flux.
  * Lit ligne par ligne (`iter_lines`) et gère 2 formats possibles :

    * lignes qui commencent par `data: {...}` + `data: [DONE]`
    * JSON direct sans `data:`
  * Extrait le texte généré principalement depuis :

    * `choices[0].delta.content` (stream OpenAI)
  * Sinon fallback sur :

    * `choices[0].message.content` (réponse non-stream)
  * Renvoie progressivement les morceaux de texte via `yield` (générateur).


### 5 - Prompts système : mode RAG strict (extraction) + mode fallback

In [6]:
SYSTEM_PROMPT_STRICT = """Tu es un assistant RAG en mode EXTRACTION PURE + MISE EN FORME.

RÈGLES ABSOLUES:
1) Tu dois UNIQUEMENT copier-coller des passages EXACTS présents dans les SOURCES.
2) Interdit: reformuler, expliquer, résumer, compléter, déduire ou corriger le texte.
3) Chaque extrait doit avoir une citation [1], [2], etc.
4) Si aucune information exacte dans les SOURCES ne répond: réponds EXACTEMENT
   "❌ Information non disponible dans mes documents."
5) N'ajoute PAS de section Sources à la fin (le serveur gère ça).
6) Maximum 6 extraits.

FORMAT:
- 1 phrase par ligne (court).
- Tu peux garder des lignes "titre:" si elles existent dans les sources.
- Pas de tableaux, pas de guillemets ajoutés.
- Si tu as "X : - A - B", mets chaque "- ..." sur une nouvelle ligne.
"""

SYSTEM_PROMPT_FALLBACK = """Tu es un assistant.
- Réponds en français.
- Si des SOURCES sont fournies, cite [1], [2], etc.
- Sinon réponds avec connaissances générales.
"""

#### Définition (points clairs)

* `SYSTEM_PROMPT_STRICT` :

  * Force un mode **RAG “extraction pure”** : uniquement copier-coller du texte présent dans les **SOURCES**.
  * Interdit toute reformulation/explication/résumé.
  * Exige des **citations** `[1]`, `[2]` pour chaque extrait.
  * Si rien n’est trouvé dans les sources → réponse fixe : `❌ Information non disponible dans mes documents.`
  * Impose un format : **phrases courtes ligne par ligne**, pas de tableaux, max 6 extraits, et transforme les listes `- ...` en lignes séparées.
* `SYSTEM_PROMPT_FALLBACK` :

  * Mode normal : répondre en **français**.
  * Si des sources existent → citer `[1]`, `[2]`.
  * Sinon → répondre avec des connaissances générales.


### 6 - Pipeline RAG “extraction stricte” : nettoyage, contexte, citations, mise en forme, sources

In [7]:
def clean_text(text: str) -> str:
    if not text:
        return text

    if any(x in text for x in ("Ã", "Â", "â€", "â€™", "â€œ", "â€")):
        try:
            fixed = text.encode("latin1", errors="ignore").decode("utf-8", errors="ignore")
            if fixed:
                text = fixed
        except Exception:
            pass

    text = (text
            .replace("\ufeff", "")
            .replace("\u200b", "")
            )

    text = re.sub(r"[ \t]+", " ", text)
    text = re.sub(r"\n{3,}", "\n\n", text)
    return text.strip()


def build_context(retrieved) -> str:
    if not retrieved:
        return "SOURCES:\n(Aucune source trouvée)\n"

    lines = ["SOURCES:\n"]
    for r in retrieved:
        filename = Path(r.chunk.meta.get("path", "unknown")).name
        page = r.chunk.meta.get("page", None)
        page_str = f"page {page}" if page else "toutes pages"
        chunk = clean_text(r.chunk.text)
        lines.append(f"[{r.ref_id}] {filename} ({page_str})\n{chunk}\n")
    return "\n".join(lines)


def run_llm(messages, max_tokens: int = 800, temperature: float = 0.0) -> str:
    out: List[str] = []
    for tok in chat_stream(messages, max_tokens=max_tokens, temperature=temperature):
        out.append(tok)
    return clean_text("".join(out).strip())


CIT_RE = re.compile(r"\[(\d{1,2})\]")
CIT_END_RE = re.compile(r"\[(\d{1,2})\]\s*$")
TITLE_RE = re.compile(r"^.{2,160}:\s*$")
QUOTED_LINE_RE = re.compile(r'^\s*["“](.*?)["”]\s*(\[\d{1,2}\])\s*$')


def extract_cited_ids(answer: str) -> List[int]:
    ids = re.findall(r"\[(\d{1,2})\]", answer)
    uniq: List[int] = []
    for x in ids:
        i = int(x)
        if i not in uniq:
            uniq.append(i)
    return uniq


def _strip_line_quotes(line: str) -> str:
    m = QUOTED_LINE_RE.match(line.strip())
    if not m:
        return line.strip()
    return f"{m.group(1).strip()} {m.group(2)}".strip()


def _split_colon_dash_to_bullets(text: str) -> str:
    text = text.replace(": -", " :\n- ")
    text = re.sub(r"\s-\s(?=[A-ZÉÈÀÂÎÔÙÇ])", "\n- ", text)
    return text


def _norm_for_match(s: str) -> str:
    s = clean_text(s)
    s = s.lower()
    s = s.replace("’", "'")
    s = re.sub(r"\s+", " ", s)
    return s.strip()


def _remove_trailing_citation(line: str) -> str:
    return re.sub(r"\s*\[\d{1,2}\]\s*$", "", line).strip()


def _find_best_ref_id_for_line(line_no_cit: str, retrieved) -> Optional[int]:
    target = _norm_for_match(line_no_cit)
    if not target:
        return None

    for r in retrieved:
        chunk = _norm_for_match(r.chunk.text)
        if target in chunk:
            return int(r.ref_id)
    return None


def _ensure_citation(line: str, retrieved, strict: bool) -> Optional[str]:
    line = line.strip()
    if not line:
        return None

    if CIT_END_RE.search(line):
        return line

    base = _remove_trailing_citation(line)
    ref_id = _find_best_ref_id_for_line(base, retrieved)
    if ref_id is None:
        return None if strict else line
    return f"{base} [{ref_id}]"


def normalize_extraction_markdown(raw: str) -> str:
    if not raw:
        return raw

    raw = clean_text(raw)
    if raw == "❌ Information non disponible dans mes documents.":
        return raw

    raw = _split_colon_dash_to_bullets(raw)

    lines = []
    for ln in raw.splitlines():
        ln2 = _strip_line_quotes(ln)
        if ln2.strip():
            lines.append(ln2.strip())
    return "\n".join(lines).strip()


def structure_nested_markdown(answer: str, retrieved, strict: bool) -> str:
    answer = normalize_extraction_markdown(answer)
    if answer == "❌ Information non disponible dans mes documents.":
        return answer

    lines = [ln.strip() for ln in answer.splitlines() if ln.strip()]
    if not lines:
        return "❌ Information non disponible dans mes documents."

    if lines and lines[0].lower().startswith("réponse extraite"):
        lines = lines[1:]

    blocks: List[Dict[str, Any]] = []
    current_title: Optional[str] = None
    current_items: List[str] = []

    def flush():
        nonlocal current_title, current_items
        if current_title is None and not current_items:
            return
        items_ok: List[str] = []
        for it in current_items:
            it = it.strip()
            if it.startswith("- "):
                it = it[2:].strip()
            it2 = _ensure_citation(it, retrieved, strict=strict)
            if it2:
                items_ok.append(it2)

        if items_ok:
            blocks.append({"title": current_title, "items": items_ok})

        current_title = None
        current_items = []

    for ln in lines:
        if TITLE_RE.match(ln) and not CIT_END_RE.search(ln) and not ln.startswith("- "):
            flush()
            current_title = ln
            continue
        current_items.append(ln)

    flush()

    if strict and not blocks:
        return "❌ Information non disponible dans mes documents."

    out: List[str] = ["**Réponse extraite**"]

    for b in blocks:
        title = b["title"]
        items = b["items"]

        if title:
            out.append(f"- **{title}**")
            for it in items:
                out.append(f"  - {it}")
        else:
            for it in items:
                out.append(f"- {it}")

    return "\n".join(out).strip()


def format_sources_section(retrieved, cited_ids: List[int]) -> str:
    if not retrieved:
        return "\n\n---\n\n📚 **Sources consultées**\n\n❌ Aucune source disponible\n"

    by_id = {r.ref_id: r for r in retrieved}
    ids_cited = [i for i in cited_ids if i in by_id]

    lines = ["\n\n---\n\n📚 **Sources consultées**\n"]
    if ids_cited:
        for i in ids_cited:
            r = by_id[i]
            filename = Path(r.chunk.meta.get("path", "unknown")).name
            page = r.chunk.meta.get("page", None)
            page_str = f"page {page}" if page else "toutes pages"
            lines.append(f"- **[{i}]** `{clean_text(filename)}` ({page_str})")
    else:
        for r in retrieved:
            filename = Path(r.chunk.meta.get("path", "unknown")).name
            page = r.chunk.meta.get("page", None)
            page_str = f"page {page}" if page else "toutes pages"
            lines.append(f"- **[{r.ref_id}]** `{clean_text(filename)}` ({page_str})")

    return "\n".join(lines) + "\n"


def answer_with_rag(retriever: Retriever, question: str, topk: int = 6, strict: bool = True) -> Dict[str, Any]:
    question = clean_text(question)
    retrieved = retriever.search(question, topk=topk)

    if strict and not retrieved:
        return {"answer": "❌ Information non disponible dans mes documents.", "retrieved": []}

    context = build_context(retrieved)

    user_prompt = f""" QUESTION: {question}
                        {context}

                        CONSIGNE:
                        - Extrais uniquement des passages EXACTS des SOURCES (copier-coller).
                        - 1 phrase par ligne.
                        - Chaque ligne doit finir par [id] (ex: [1]).
                        - Interdit: reformuler/expliquer.
                        - Si tu as "X : - A - B", mets chaque "- ..." sur une nouvelle ligne.
                        - Si rien: "❌ Information non disponible dans mes documents."
                    """

    messages = [
        {"role": "system", "content": SYSTEM_PROMPT_STRICT if strict else SYSTEM_PROMPT_FALLBACK},
        {"role": "user", "content": user_prompt},
    ]

    raw = run_llm(messages, max_tokens=900, temperature=0.0)
    answer = structure_nested_markdown(raw, retrieved, strict=strict)

    cited = extract_cited_ids(answer)
    if strict and answer != "❌ Information non disponible dans mes documents." and not cited:
        answer = "❌ Information non disponible dans mes documents."
        cited = []

    sources_section = format_sources_section(retrieved, cited)
    final_answer = answer + sources_section

    return {"answer": final_answer, "retrieved": retrieved}

#### Définition (points clairs)

* **Nettoyage texte** (`clean_text`) : corrige quelques problèmes d’encodage (latin1→utf8), supprime caractères invisibles, normalise espaces et sauts de ligne.
* **Construction du contexte** (`build_context`) : assemble une section `SOURCES:` avec chaque chunk récupéré + infos fichier/page + id `[1]`, `[2]`…
* **Appel LLM** (`run_llm`) : consomme le streaming `chat_stream`, concatène les tokens, puis nettoie le résultat.
* **Regex utilitaires** : détecte citations `[n]`, titres “xxx:”, lignes entre guillemets, etc.
* **Gestion des citations** :

  * `extract_cited_ids` récupère les ids cités dans la réponse.
  * `_ensure_citation` ajoute une citation manquante en cherchant si la ligne existe dans un chunk (matching simple “ligne ⊂ chunk”).
  * En mode `strict`, une ligne sans source trouvée peut être supprimée.
* **Normalisation du markdown extrait** :

  * `normalize_extraction_markdown` enlève guillemets, restructure les listes du type `X : - A - B` en puces sur plusieurs lignes.
  * `structure_nested_markdown` regroupe en blocs : titres en **gras** + sous-puces, et garantit (autant que possible) une citation par ligne.
* **Section “Sources consultées”** (`format_sources_section`) :

  * Affiche uniquement les sources réellement citées si possible, sinon liste toutes les sources récupérées.
* **Fonction principale** (`answer_with_rag`) :

  * Nettoie la question → `retriever.search(topk)`
  * Si `strict` et rien trouvé → renvoie directement `❌ Information non disponible...`
  * Construit un prompt utilisateur (question + SOURCES + consignes strictes)
  * Appelle le LLM, restructure la réponse, vérifie qu’il y a des citations (sinon refuse en strict)
  * Ajoute la section sources et renvoie `{ "answer": ..., "retrieved": ... }`.


### 7 - Initialisation RAG : chargement .env + build/load index + création du Retriever

In [8]:
load_dotenv()

print("🔄 Chargement index RAG...")
rag_index = build_or_load_index(data_dir=DATA_DIR, cache_dir=CACHE_DIR)
retriever = Retriever(rag_index)
print("✅ Index prêt.")

🔄 Chargement index RAG...


Scan & Cache: 100%|██████████| 7/7 [00:02<00:00,  2.62it/s]


✅ Index prêt.


#### Définition (points clairs)

* `load_dotenv()` : charge les variables d’environnement depuis le fichier `.env` (ex: clé API, modèle, etc.).
* Construit/charge l’index RAG avec `build_or_load_index(DATA_DIR, CACHE_DIR)` (scan fichiers, chunks, embeddings, cache, FAISS).
* Crée `retriever = Retriever(rag_index)` pour pouvoir faire des recherches top-k dans l’index.
* Affiche des messages console pour indiquer le début et la fin du chargement (`Index prêt`).


### 8 - Question RAG : recherche + extraction stricte + affichage de la réponse

In [13]:
q = "Le Model Context Protocol ?"
res = answer_with_rag(retriever, q, topk=6, strict=True)
print(res["answer"])

**Réponse extraite**
- Le Model Context Protocol (MCP) est un protocole standard ouvert qui permet de connecter facilement un modèle d’intelligence artificielle (comme ChatGPT, Claude ou Gemini) à des outils, des services et des sources de données externes. [2]
- L’idée principale est la suivante : au lieu de créer une intégration différente pour chaque modèle d’IA et chaque service [2]

---

📚 **Sources consultées**

- **[2]** `01_definition_mcp.txt` (toutes pages)



#### Définition (points clairs)

* Définit la question `q` (“Le Model Context Protocol ?”).
* Appelle `answer_with_rag(...)` avec :

  * `topk=6` : récupère jusqu’à 6 chunks les plus proches.
  * `strict=True` : réponse **uniquement par copier-coller** des sources, avec citations obligatoires.
* Récupère le texte final dans `res["answer"]` (réponse + “Sources consultées”).
* Affiche le résultat avec `print(...)`.


In [ ]:
q = "Le nom du roi du Maroc en 2020 ?"
res = answer_with_rag(retriever, q, topk=6, strict=True)
print(res["answer"])

❌ Information non disponible dans mes documents.

---

📚 **Sources consultées**

- **[1]** `05_exemples_utilisation_mcp.txt` (toutes pages)
- **[2]** `06_avantages_limites_mcp.txt` (toutes pages)
- **[3]** `01_definition_mcp.txt` (toutes pages)
- **[4]** `04_fonctionnement_mcp.txt` (toutes pages)
- **[5]** `07_sans_mcp_vs_avec_mcp.txt` (toutes pages)
- **[6]** `02_objectifs_mcp.txt` (toutes pages)
- **[7]** `06_avantages_limites_mcp.txt` (toutes pages)
- **[8]** `07_sans_mcp_vs_avec_mcp.txt` (toutes pages)

